In [1]:
#here are some useful imports for this particular series of examples 
from scipy.optimize import linprog
from IPython.core.display import Image, display
! pip install pulp
from __future__ import division
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy

Defaulting to user installation because normal site-packages is not writeable


Matplotlib created a temporary config/cache directory at /tmp/matplotlib-v8l5pc0f because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# An industrial bakery: from the grain to your door

Our problem focuses around an industrial bakery and how to optimize the production and deliver to minimize cost and maximize gain. From this general statement, we envision four different automation areas:
1. Production optimization: focused on the factories themselves, we want to decide when and how to produce certain products. We formulate the problem as a decision of how many days we need each factory to produce each product, in order to meet the clients' requirements. This is a classic example of Linear Programming, where we need to account for the supply and demand constraints to obtain, not only a feasible solution, but also an optimized one.
2. Product fitting: once we have the products, we need to fit them products into a truck. Since the objective is to maximize revenue, we want to fit the maximum value into a volume-limited truck. This is a classic instance of a Knapsack problem, which is a subclass of the more general Constrained Problems.
3. Vehicle routing: once the truck is filled, the objective is to bring the products to the clients using the most efficient route. This is a classic instance of the travel salesman problem, where we need to visit certain locations in order to sell the products.
4. Truck diagnosis and repairing: since trucks are a fundamental factor in our business case, we need to ensure that we can diagnose, at any point, the state of the truck and be able to determine a correct repairing procedure to avoid unnecessary delays.

Each of the following sections details each one of the specific sub-problems. 

The main contributors of each sections are: Alex Hillman and Skylar Eiskowitz (1), Celina Pasiecznik (2), Julia Pasiecznik (3), and Nils Pachler de la Osa (4).

# 4. Truck diagnosis and repairing
The objective of this section is to provide a tool for truck diagnosis as well as a tool for automated decision making for truck maintenance, according to the diagnosis. Throughout the section, we consider a general truck of any dimensions and manufacturer. Thus, the failure probabilities are based on the authors' best estimation, and should be refined for a better diagnosis with the help of a manufacturer or an expert.

To be able to diagnose the state of a truck, we need to understand what components form the truck. Just as a toy example, we will assume that a truck is composed of 10 different components. Each component can be in different states, according to the code below.

In [1]:
TruckStates = {
    'Breaks': ['OK', 'AlmostBroken', 'Broken', 'Unknown'],
    'Engine': ['OK', 'Overheated', 'Broken', 'Unknown'],
    'Starter': ['OK', 'Broken', 'Unknown'],
    'U-Joint': ['OK', 'AlmostBroken', 'Broken', 'Unknown'],
    'Tires': ['OK', 'Broken', 'Unknown'],  # Broken == Flat tire
    'Fuel': ['Empty', 'Not Empty'],
    'Battery': ['Empty', 'Not Empty'],
    'EngineSensor': ['OK', 'Not OK'],
    'FuelSensor': ['OK', 'Not OK'],
    'BatterySensor': ['OK', 'Not OK']
}

Each component has an independent failure probability that will depend on multiple factors: manufacturer, average load weight, state of the road, experience of the driver, etc. While this numbers may be highly volatile, we will assume the fixed values specified below. These numbers can be further refined with more information of the specific operations environment.

In [2]:
TruckStatesProbabilities = {
    'Breaks': [1 - 0.05 - 0.02 - 0.001, 0.05, 0.02, 0.001],
    'Engine': [1 - 0.05 - 0.01 - 0.001, 0.05, 0.01, 0.001],
    'Starter': [1 - 0.04 - 0.001, 0.04, 0.001],
    'U-Joint': [1 - 0.05 - 0.03 - 0.001, 0.05, 0.03, 0.001],
    'Tires': [1 - 0.01 - 0.001, 0.01, 0.001],  # Broken == Flat tire
    'Fuel': [0.1, 0.9],
    'Battery': [0.11, 0.89],
    'EngineSensor': [0.9999, 0.0001],
    'FuelSensor': [0.9999, 0.0001],
    'BatterySensor': [0.9999, 0.0001]
}

For any combination of states, the driver will experience a set of symptoms with which we can infer the real state of the truck. For example, if driving is smooth in both highway and city, it is highly likely that all the components of the truck are in a correct state. If something fails, it is likely that the driver will see or feel something unusual. For our example, we will work with 6 possible self-descriptive symptoms:

In [ ]:
TruckSymptoms = {
    'HighwayDriving': ['Normal', 'Not high speeds', 'Impossible', 'Unknown'],  # Feeling when driving in highways
    'CityDriving': ['Normal', 'Slow break', 'Impossible', 'Unknown'],  # Feeling when driving in cities
    'EngineStart': ['Yes', 'No', 'Unknown'],  # Does the engine start?
    'FuelLevel': ['Empty', 'Not Empty', 'Unknown'],  # What's the fuel level, according to visual indicators
    'BatteryLevel': ['Empty', 'Not Empty', 'Unknown'],  # What's the battery level, according to visual indicators
    'EngineTemperature': ['OK', 'Not OK', 'Unknown']  # What's the engine temperature, according to visual indicators
}

Each observed symptom indicates certain knowledge about the possible states of the truck. For example, if driving in highway is smooth, we know that the engine must be OK (otherwise we could not reach higher speeds, or even ignite the engine), the U-Joint must be OK (otherwise we could not reach higher speeds, or even drive at all), the tires must be OK (otherwise we could not reach higher speeds, or even drive at all), and we must have some fuel left. The same logic can be applied to every symptom to create a propositional model that validates the state of the truck. The specific logic used in our example is determined in the code below.

In [3]:
TruckClauses = [
    [['HighwayDriving', 'is', 'Normal'], 'implies', ['Engine', 'is', 'OK']],
    [['HighwayDriving', 'is', 'Normal'], 'implies', ['U-Joint', 'is', 'OK']],
    [['HighwayDriving', 'is', 'Normal'], 'implies', ['Tires', 'is', 'OK']],
    [['HighwayDriving', 'is', 'Normal'], 'implies', ['Fuel', 'is', 'Not Empty']],

    [['HighwayDriving', 'is', 'Not high speeds'], 'implies', [['Engine', 'is', 'OK'], 'or', ['Engine', 'is', 'Overheated']]],
    [['HighwayDriving', 'is', 'Not high speeds'], 'implies', [['U-Joint', 'is', 'OK'], 'or', ['U-Joint', 'is', 'AlmostBroken']]],
    [['HighwayDriving', 'is', 'Not high speeds'], 'implies', [['Tires', 'is', 'OK'], 'or', ['Tires', 'is', 'Broken']]],
    [['HighwayDriving', 'is', 'Not high speeds'], 'implies', ['Fuel', 'is', 'Not Empty']],
    [['HighwayDriving', 'is', 'Not high speeds'], 'implies',
        [['Engine', 'is', 'Overheated'], 'or', [['U-Joint', 'is', 'AlmostBroken'], 'or', ['Tires', 'is', 'AlmostBroken']]]],

    [['HighwayDriving', 'is', 'Impossible'], 'implies',
        [['Engine', 'is', 'Broken'], 'or', [['U-Joint', 'is', 'Broken'], 'or', ['Fuel', 'is', 'Empty']]]],

    [['CityDriving', 'is', 'Normal'], 'implies', [['Engine', 'is', 'OK'], 'or', ['Engine', 'is', 'Overheated']]],
    [['CityDriving', 'is', 'Normal'], 'implies', [['U-Joint', 'is', 'OK'], 'or', ['U-Joint', 'is', 'AlmostBroken']]],
    [['CityDriving', 'is', 'Normal'], 'implies', ['Tires', 'is', 'OK']],
    [['CityDriving', 'is', 'Normal'], 'implies', ['Fuel', 'is', 'Not Empty']],
    [['CityDriving', 'is', 'Normal'], 'implies', ['Breaks', 'is', 'OK']],

    [['CityDriving', 'is', 'Slow break'], 'implies', [['Engine', 'is', 'OK'], 'or', ['Engine', 'is', 'Overheated']]],
    [['CityDriving', 'is', 'Slow break'], 'implies', [['U-Joint', 'is', 'OK'], 'or', ['U-Joint', 'is', 'AlmostBroken']]],
    [['CityDriving', 'is', 'Slow break'], 'implies', [['Tires', 'is', 'OK'], 'or', ['Tires', 'is', 'Broken']]],
    [['CityDriving', 'is', 'Slow break'], 'implies', ['Fuel', 'is', 'Not Empty']],
    [['CityDriving', 'is', 'Slow break'], 'implies', [['Breaks', 'is', 'OK'], 'or', ['Breaks', 'is', 'AlmostBroken']]],
    [['CityDriving', 'is', 'Slow break'], 'implies', [['Tires', 'is', 'Broken'], 'or', ['Breaks', 'is', 'AlmostBroken']]],

    [['CityDriving', 'is', 'Impossible'], 'implies',
        [['Engine', 'is', 'Broken'], 'or', [['U-Joint', 'is', 'Broken'],
         'or', [['Fuel', 'is', 'Empty'], 'or', ['Breaks', 'is', 'Broken']]]]],

    [['EngineStart', 'is', 'Yes'], 'implies', [['Engine', 'is', 'OK'], 'or', ['Engine', 'is', 'Overheated']]],
    [['EngineStart', 'is', 'Yes'], 'implies', ['Starter', 'is', 'OK']],
    [['EngineStart', 'is', 'Yes'], 'implies', ['Fuel', 'is', 'Not Empty']],
    [['EngineStart', 'is', 'Yes'], 'implies', ['Battery', 'is', 'Not Empty']],

    [['EngineStart', 'is', 'No'], 'implies',
        [['Engine', 'is', 'Broken'], 'or', [['Starter', 'is', 'Broken'],
         'or', [['Fuel', 'is', 'Empty'], 'or', ['Battery', 'is', 'Empty']]]]],

    [['FuelLevel', 'is', 'Empty'], 'implies', [['Fuel', 'is', 'Empty'], 'or', ['FuelSensor', 'is', 'Not OK']]],
    [['FuelLevel', 'is', 'Not Empty'], 'implies', [['Fuel', 'is', 'Not Empty'], 'or', ['FuelSensor', 'is', 'Not OK']]],

    [['BatteryLevel', 'is', 'Empty'], 'implies', [['Battery', 'is', 'Empty'], 'or', ['BatterySensor', 'is', 'Not OK']]],
    [['BatteryLevel', 'is', 'Not Empty'], 'implies', [['Battery', 'is', 'Not Empty'], 'or', ['BatterySensor', 'is', 'Not OK']]],

    [['EngineTemperature', 'is', 'OK'], 'implies', [['Engine', 'is', 'OK'], 'or', ['EngineSensor', 'is', 'Not OK']]],
    [['EngineTemperature', 'is', 'Not OK'], 'implies', [['Engine', 'is', 'Overheated'], 'or', [['Engine', 'is', 'Broken'], 'or', ['EngineSensor', 'is', 'Not OK']]]]
]

Finally, once we determine the state of the truck, we have different repairing options to choose from. Although a full check makes sure everything is OK, it is usually more expensive and may not always be needed. For example, if we determined that the most likely cause of the symptoms is a flat tire, we can save some money by only exchanging the tires, instead of running a full check. The set of options, as well as the cost of each option, is summarized in the code below.

In [4]:
TruckRepairing = {  # Effect of each repairing option
    'BreakCheck': {'Breaks': 'OK'},
    'TireCheck': {'Tires': 'OK'},
    'EngineCheck': {'Starter': 'OK', 'Engine': 'OK'},
    'ConventionalCheck': {'Breaks': 'OK', 'Tires': 'OK', 'Starter': 'OK', 'EngineSensor': 'OK'},
    'TransmissionCheck': {'Starter': 'OK', 'Engine': 'OK', 'U-Joint': 'OK'},
    'FullCheck': {'Breaks': 'OK', 'Tires': 'OK', 'Engine': 'OK', 'U-Joint': 'OK',
                  'Starter': 'OK', 'FuelSensor': 'OK', 'BatterySensor': 'OK', 'EngineSensor': 'OK'},
    'SensorCheck': {'FuelSensor': 'OK', 'BatterySensor': 'OK', 'EngineSensor': 'OK'},
    'GasStationCheck': {'Fuel': 'Not Empty', 'Battery': 'Not Empty'}
}

TruckCosts = {  # Cost of each repairing option
    'BreakCheck': 80,
    'TireCheck': 40,
    'EngineCheck': 100,
    'ConventionalCheck': 200,
    'TransmissionCheck': 400,
    'FullCheck': 500,
    'SensorCheck': 20,
    'GasStationCheck': 40
}

Just as a helper class, we implemented a simple version of a priority queue that will be used later on.

In [ ]:
class PriorityQueue:
    """
    Simple class to implement a Priority queue. It contains a list sorted on descending order
    """
    def __init__(self):
        """
        Initialization of the list as an empty list
        """
        self.queue = []

    def add(self, node, value):
        """
        Function to add an element to the queue
        :param node: Node to be added
        :param value: Value of the node
        :return: -
        """
        def add_recursive(self, node, value, left, right):
            """
            Recursive function to add an element to the queue. It uses the fact that the list is already ordered
            :param self: PriorityQueue element containing the list
            :param node: Node to add
            :param value: Value of the node
            :param left: Left index to look at
            :param right: Right index to look at
            :return: -
            """
            if right - left <= 0:
                if right == len(self.queue):
                    self.queue.append((value, node))
                else:
                    self.queue.insert(right, (value, node))
                return
            m = int((left + right) / 2)
            if self.queue[m][0] > value:
                add_recursive(self, node, value, m + 1, right)
            else:
                add_recursive(self, node, value, left, m)
        add_recursive(self, node, value, 0, len(self.queue))

    def pop(self):
        """
        Function to pop the first element of the queue
        :return: Returns the node and the value of that node
        """
        aux = self.queue.pop(0)
        return aux[1], aux[0]

    def __len__(self):
        """
        :return: Returns the length of the queue
        """
        return len(self.queue)

Now, given the described model and a set of symptoms, we need to develop the code to determine the most likely set of states that the truck can be in. Before that, we can observe that our model is not purely written in propositional clauses (due to the "implies" logic). Therefore, we will develop code to check if the Model is a correctly formatted Model, and code to transform the Model into a set of propositional clauses.

In [5]:
def check_clause(clause, States, Symptoms):
    """
    Function to check if the data structure of the clause or any sub-clause is correct
    :param clause: Clause to be checked, to be correct must contain three elements, if the second element is either
    "is" or "is not" the first and third elements must be strings containing an element assignment, otherwise the first
    and third clauses must be sub-clauses themselves
    :param States: Dictionary with the possible states of the elements in the model (not including symptoms)
    :param Symptoms: Dictionary with the possible symptoms of the elements in the model (not including states)
    :return: Returns True if the clause and sub-clauses are valid, False otherwise
    """
    raise NotImplementedError
    
    
def check_logic(clauses, States, Symptoms):
    """
    Function to check the format validity of a Model
    :param clauses: Clauses to be checked
    :param States: Dictionary with the possible states of the elements in the model (not including symptoms)
    :param Symptoms: Dictionary with the possible symptoms of the elements in the model (not including states)
    :return: Returns False if any of the clauses is not properly formatted
    """
    raise NotImplementedError

    
def transform_logic(clause, negate=False):  
    """
    Function to transform a specific clause into a propositional logic clause
    :param clause: Clause to be transformed, must contain exactly three elements
    :param negate: Boolean to determine if we need to negate the clause or not
    :return: Returns a list of propositional clauses
    """
    raise NotImplementedError

    
def transform_model(Model, States, Symptoms):  
    """
    Function to transform an entire Model into a set of propositional clauses
    :param Model: Model to be transformed, needs to be a list of clauses
    :param States: Dictionary with the possible states of the elements in the model (not including symptoms)
    :param Symptoms: Dictionary with the possible symptoms of the elements in the model (not including states)
    :return: Returns a new Model as a set of clauses on propositional logic with only: OR, IS, and IS NOT.
    """
    raise NotImplementedError

Although we have the necessary code to transform the Model into a set of propositional clauses, we still need code to check if a specific set of states is valid given a set of observations.

In [6]:
def validate_clause(clause, node, current_symptoms):
    """
    Function to validate a specific set of states against a clause given a set of observations
    :param clause: Clause to validate against
    :param node: Set of states to validate
    :param current_symptoms: Observed symptoms
    :return: True if the clause is satisfied, False otherwise
    """
    raise NotImplementedError
    
    
def read_conflict(clause, States, current_symptoms, negate=False):
    """
    Function to read the conflict underlying a clause
    :param clause: Clause to read the conflict from
    :param States: Dictionary with the possible states of the elements in the model (not including symptoms)
    :param current_symptoms: Observed symptoms
    :param negate: Boolean to determine if we need to negate the clause or not
    :return: Returns a set of possible states that are necessary to satisfy the clause
    """
    raise NotImplementedError

    
def validate_state(clauses, node, current_symptoms, States):
    """
    Function to validate a specific set of states against a Model given a set of observations
    :param clauses: Model that we want to validate against (must be a valid Model)
    :param node: Set of states that we want to check
    :param current_symptoms: Observed symptoms
    :param States: Dictionary with the possible states of the elements in the model (not including symptoms)
    :return: Returns True if the set of states is valid within a Model. False otherwise.
    In addition, if the result is False, it returns one conflict that the current set does not satisfy
    """
    raise NotImplementedError

Now, before going to the core of the diagnosis, our propositional logic functions rely on detecting what are the sources of conflicts and finding kernel diagnoses to work with. Thus, we need to define helper functions that add conflicts to current kernel diagnoses.

In [7]:
def check_is_subset(s1, s2):
    """
    Function to check if the dictionary of s2 is a subset of s1
    :param s1: Dictionary (i.e., the superset)
    :param s2: Dictionary (i.e., the subset)
    :return: Returns True if s2 is a subset of s1, False otherwise
    """
    raise NotImplementedError

    
def ComputeProbability(s1, States, Probabilities):
    """
    Function to compute the probability of a specific set of states
    :param s1: Set of states to compute the probability against
    :param States: Dictionary with the possible states of the elements in the model (not including symptoms)
    :param Probabilities: Dictionary with the probability value of each state
    :return: Returns the probability of the set s1
    """
    raise NotImplementedError
    
    
def update_kernel_diagnoses(kernel_diagnoses, conflict, States, Probabilities):
    """
    Function to update a kernel diagnosis given a new conflict
    :param kernel_diagnoses: Current set of kernel diagnoses
    :param conflict: New conflict found
    :param States: Dictionary with the possible states of the elements in the model (not including symptoms)
    :param Probabilities: Dictionary with the probability value of each state
    :return: Returns the updated list of kernel diagnoses including the conflict
    """
    raise NotImplementedError

Finally, we can describe the function that performs a best-first search towards the most probable configurations. Note that we are talking about configurations in plural because we want a probability estimate for each one. Each function will have the capability of either inputing how many diagnoses to return, or the desired threshold probability. As mentioned, the logic of the tree relies on kernel diagnoses and conflicts: once a specific state is reached, if the state is valid for the model, it is added to the return list and, if not, the underlying conflict is extracted and the whole search is restarted with the new information. Although this may seem to add innecessary computation, in reality the algorithm runs much faster due to the early pruning of invalid options.

In [ ]:
def LikelihoodSearchTree(initial_tree, current_symptoms, Model, States, Probabilities, N, P):
    """
    Function to perform the best-first search based on probabilities and kernel diagnoses
    :param initial_tree: Initial tree extracted from previous kernel diagnoses
    :param current_symptoms: Observed symptoms
    :param Model: Model to validate against
    :param States: Dictionary with the possible states of the elements in the model (not including symptoms)
    :param Probabilities: Dictionary with the probability value of each state
    :param N: Number of valid states to retrieve. If different than None, this has priority over P
    :param P: Threshold probability of the desired valid states to retrieve (i.e., we want the set of valid states to 
    represent a probability of at least P)
    :return: Returns a set of valid states
    """
    raise NotImplementedError
    

def FindMostLikelyStates(Statement, Model, States, Probabilities, Symptoms, N=None, P=None):
    """
    Function to validate the Model and perform the search
    :param Statement: Observed symptoms
    :param Model: Model to validate against
    :param States: Dictionary with the possible states of the elements in the model (not including symptoms)
    :param Probabilities: Dictionary with the probability value of each state
    :param Symptoms: Dictionary with the possible symptoms of the elements in the model (not including states)
    :param N: If given, maximum number of valid states to return
    :param P: If given, threshold probability of the set of valid states
    :return: Returns the set of valid states with maximum probability according to the symptoms
    """
    raise NotImplementedError

The previous functions return the set of most probable diagnoses according to a set of observed symptoms. However, when the possible diagnoses is large (due to diagnoses with similar probabilities, or a large confidence value), reading through all of them and determining the best course of action might be a nightmare. The objective of the next part is to automatically determine the best course of action given a set of diagnoses and their probability, a set of repairing actions, a goal state, and a confidence probability value. In plain words, we want that, not matter what the actual correct diagnoses is, to ensure that after the repairs we get to the desired state with at least a given probability. This can be done using a best-first search where now the cost value is the cost of the repairings.

In [9]:
def CheckAllStates(states, goal):
    """
    Function to check if a specific list of states satisfies the goal state
    :param states: List of states to check
    :param goal: State to satisfy
    :return: Returns True if all the states satisfy the goal, False otherwise
    """
    raise NotImplementedError
    

def CheckIfDone(action_list, visited_list):
    """
    Helper function to determine if a specific set of actions have already been performed. This is just to avoid revisit
    :param action_list: Set of actions to check
    :param visited_list: List of sets of actions already visited
    :return: True if the action set has already been visited, False otherwise
    """
    raise NotImplementedError
    
    
def FilterState(possible_states, prob):
    """
    Function to filter the possible states according to a given probability
    :param possible_states: Possible valid states to filter
    :param prob: Probability to filter against
    :return: Returns a new set of states with probability of at least prob
    """
    raise NotImplementedError
    

def RepairTruck(possible_states, Actions, Cost, goal):
    """
    Function to determine the course of action to ensure a successful repair
    :param possible_states: List of state sets that we need to correct
    :param Actions: Dictionary with each of the repairing actions and the effect of each one
    :param Cost: Dictionary with the cost of each repairing action
    :param goal: Set of states that determine the objective goal
    :return: Returns the list of repairing actions to take to achieve the goal
    """
    raise NotImplementedError

Now we have all the necessary elements to run our model against multiple observations and desired probabilities



In [10]:
DetectionProbability = 0.99
RepairProbability = 0.7

Statement = [['CityDriving', 'is', 'Slow break'], ['EngineStart', 'is', 'Yes'],
             ['FuelLevel', 'is', 'Not Empty'], ['BatteryLevel', 'is', 'Not Empty'], ['EngineTemperature', 'is', 'OK']]

NewModel = transform_model(TruckClauses, TruckStates, TruckSymptoms)

possible_states = FindMostLikelyStates(Statement, NewModel, TruckStates, TruckStatesProbabilities, TruckSymptoms, P=DetectionProbability)

new_states = FilterState(possible_states, prob)

goal = {'Breaks': 'OK', 'Engine': 'OK', 'Starter': 'OK', 'U-Joint': 'OK', 'Tires': 'OK', 'Fuel': 'Not Empty',
        'Battery': 'Not Empty'}

print(RepairTruck(new_states, TruckRepairing, TruckCosts, goal))

NameError: name 'TruckSymptoms' is not defined